In [1]:
import IP2Location #convert ip to latlon
import csv,json 
import pathlib
import requests
from urllib.request import urlopen
import pandas as pd
from git import Repo
import os
#DBs
import sqlite3
import pymysql
import re
import datetime
from datetime import date,timedelta
from time import strptime

r = Repo(str(pathlib.Path().resolve())+'/blacklisted_ipsets/')

plus = "^\+\d{1}"
minus = "^\-\d{1}"
IP2LocObj = IP2Location.IP2Location()
IP2LocObj.open('/home/dakes/Downloads/Jupyter_scripts/IP2LOCATION-LITE-DB5.BIN')
ff=[] 
allo=[]
x = datetime.datetime.now()
today= date(x.year,x.month,x.day)
print(today)
import pathlib
#print(pathlib.Path().resolve())

2021-07-13


In [ ]:
for remote in r.remotes:
    remote.fetch()

In [2]:
import mysql.connector


In [3]:
mydb = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    database="Blocklists"
    
    #auth_plugin='mysql_native_password'
)
mycursor= mydb.cursor()
mycursor.execute("CREATE  TABLE IF NOT EXISTS Ipsets(ip  VARCHAR(255),location VARCHAR(255),source VARCHAR(255),category VARCHAR(255),description VARCHAR(255),maintainer VARCHAR(255),alt1 VARCHAR(255),alt2 VARCHAR(255))")
#,geo POINT                                  

In [ ]:
mycursor.execute("SHOW TABLES")

for x in mycursor:
    print(x)

In [ ]:
fnames=[]
with open('blacklisted_ipsets.csv', 'w', newline='') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        spamwriter.writerow(['Name'])#,'Maintainer','Maintainer URL','Tracker List Url','Source File Date','File Update','Last update','IPs'])#+x.month*100 + x.day)
        for path in pathlib.Path("/home/dakes/blacklisted_ipsets").iterdir():
            if str(path).endswith('.netset') or str(path).endswith('.ipset'):
                df = pd.read_csv(path,sep=";ofi",index_col=False,engine='python')
                for line in df.iterrows():
                        if "# Category" in line[1][0] :
                                    if "malware" not in line[1][0] and "spam" not in line[1][0] and "attacks" not in line[1][0]:  
                                        break
                                    else:
                                        fnames.append(str(path).replace("/home/dakes/blacklisted_ipsets/",""))
                                        name = str(path).replace('/home/dakes/blacklisted_ipsets/','')
                                        spamwriter.writerow([name.replace('/home/dakes/blacklisted_ipsets/','')])#,maintainer,maintainerURL,listSourceUrl,sourceFileDate,recentFileUpdate,(x.month*100+x.day),','.join(diff_cut)])

#print((fnames))
with open("olatafiles.json","w") as f: #edit your  path
    json.dump(fnames,f,indent=4) 
f.close()
print(len(fnames))

In [28]:
        insert_ip = "INSERT INTO Ipsets (ip,location,source,category,description,maintainer ,alt1,alt2 ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
        with open("/home/dakes/description.json",'r') as jsf:
            descr = json.load(jsf,strict = False)
        files=[]
        removed=[]
    
        for path in pathlib.Path("/home/dakes/blacklisted_ipsets").iterdir():
            flag =False
            if str(path).endswith('.netset') or str(path).endswith('.ipset'):
                diff_cut=[]
                df = pd.read_csv(path,sep=";ofi",index_col=False,engine='python')
                name=''
                maintainer=''
                maintainerURL=''
                listSourceUrl=''
                sourceFileDate=''
                recentFileUpdate=''
                for line in df.iterrows():
                        if "# Category" in line[1][0] :
                            if "malware" not in line[1][0] and "spam" not in line[1][0] and "attacks" not in line[1][0]:  
                                break
                            else:
                                category = line[1][0].replace("# Category      : ","")
                        if "# Maintainer      :" in line[1][0]:
                            maintainer = line[1][0].replace("# Maintainer      : ","")
                        if "# Maintainer URL  :" in line[1][0]:
                            maintainerURL = line[1][0].replace("# Maintainer URL  : ","")
                        if "# List source URL :" in line[1][0]:
                            listSourceUrl = line[1][0].replace("# List source URL : ","")
                        if "# Source File Date:" in line[1][0]:
                            sourceFileDate = line[1][0].replace("# Source File Date: ","")
                                
                        if "# This File Date  :" in line[1][0]:
                            year=0
                            timestamp=''
                            recentFileUpdate = line[1][0].replace("# This File Date  : ","")
                            name = str(path).replace('/home/dakes/blacklisted_ipsets/','')
                            timestamp=recentFileUpdate.split(' ')
                            timestamp=list(filter(None,timestamp))
                            
                            day=int(timestamp[2])
                            month = strptime(timestamp[1],'%b').tm_mon
                            year = int(timestamp[5])

                            anyday = date(year,month,day)
                            
                            #print(today-anyday)
                            if today-anyday <=timedelta(days=40) :
                                flag=True
                            else:
                                flag=False
                                break
                            
                            
                            diff = r.git.diff("@{upstream}",name)
                            source = str(path)
                            des = source.replace("/home/dakes/blacklisted_ipsets/","")
                            des = des.replace('.ipset',"")
                            des = des.replace('.netset',"")
                            for line in diff.splitlines():
                                if re.findall(plus,line): #create today's list
                                    #print(line)
                                    line = line.split('/',1)[0]
                                    line = ".".join(line.split('.')[0:-1])
                                    line = line +'.1'
                                    diff_cut.append(line.replace('+',""))
                                    rec =  IP2LocObj.get_all(line.replace('+',""))
                                    
                                    here = (rec.ip,rec.city,listSourceUrl,category,"descr[des]",maintainer,"extreme-ip-lookup.com/"+rec.ip,"talosintelligence.com/reputation_center/lookup?search="+rec.ip)
                                    mycursor.execute(insert_ip,here)
                                    
                                elif re.findall(minus,line): #removes last day of the week and (-) of other days
                                    line = line.split('/',1)[0]
                                    line = ".".join(line.split('.')[0:-1])
                                    line = line +'.1'
                                    rec =  IP2LocObj.get_all(line.replace('-',""))
                                    
                                    deleto = "DELETE from Ipsets WHERE ip = '$rec.ip'"
                                    #print(deleto)
                                    #query=deleto+line.replace('-',"")
                                    mycursor.execute(deleto)
                                    
                            break      
                if  flag:
                    #print(flag,recentFileUpdate)
                    files.append((name,maintainer,maintainerURL,listSourceUrl,sourceFileDate,recentFileUpdate,diff_cut,descr[des]))
mydb.commit()
print(mycursor.rowcount, "record inserted.")

#print(len(files))


KeyboardInterrupt: 

In [25]:
insert_ip = "INSERT INTO Ipsets (ip,location,source,category,description,maintainer ,alt1,alt2 ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
here = []
for i in files:
    if not i[6] or i[6]=='127.0.0.1' or i[6]=='192.168.1.1':
        continue
    else:
        for ip in i[6]:
            rec =  IP2LocObj.get_all(ip)
            thisip = rec.ip#".".join(rec.ip.split('.')[0:-1])+'.*'
            thissource= "https://github.com/firehol/blocklist-ipsets/blob/master/"+i[0].replace('/home/dakes/blacklisted_ipsets/','')
            here.append((thisip,rec.city,thissource,'malware',i[7],i[1],"extreme-ip-lookup.com/"+rec.ip,"talosintelligence.com/reputation_center/lookup?search="+rec.ip))


In [ ]:
print((here[0]))
print(len(insert_ip))

In [ ]:
mycursor.executemany(insert_ip,here)

In [ ]:
features=[]
for i in files:
    if not i[6] or i[6]=='127.0.0.1' or i[6]=='192.168.1.1':
        continue
    else:
        for ip in i[6]:
            rec =  IP2LocObj.get_all(ip)
            #print(".".join(ip.split('.')[0:-1])+'.*')
            features.append(({
                                  "type": "Feature",
                                  "properties": {
                                    "location": rec.city,
                                    "source":"https://github.com/firehol/blocklist-ipsets/blob/master/"+i[0].replace('/home/dakes/blacklisted_ipsets/',''),
                                    "ip": ".".join(rec.ip.split('.')[0:-1])+'.*',
                                    "maintainer":i[1],
                                    "category" : "malware",
                                    "description": i[7],
                                    "alt1": "extreme-ip-lookup.com/"+rec.ip,
                                    "alt2":"talosintelligence.com/reputation_center/lookup?search="+rec.ip
                                    #"0datetime":i[7]
                                  },
                                  "geometry": {
                                    "type": "Point",
                                    "coordinates": [float(rec.longitude), float(rec.latitude)]
                                  }
                                }))
#len(features)
#del features[1000:]
#len(features)
